In [1]:
import numpy as np
import pandas as pd

import os
#os.sys.path
import sys
sys.path.append('../src')

from pathlib import Path

from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as imgplt
import code1_data as go

In [2]:
inp1_path = Path.home()/'Iron'/'inp1'

train_path = inp1_path/'train'

testvalidation_path = inp1_path/'test_validation'

fer_csv = inp1_path/'Fer.csv'

train_csv = train_path/'Training_Data.csv'

test_csv = testvalidation_path/'Testing_Data.csv'

validation_csv = testvalidation_path/'Validation_Data.csv'


In [3]:
df_fer = pd.read_csv(fer_csv,encoding = "ISO-8859-1")
df_train = pd.read_csv(train_csv,encoding = "ISO-8859-1")
df_test = pd.read_csv(test_csv,encoding = "ISO-8859-1")
df_val = pd.read_csv(validation_csv,encoding = "ISO-8859-1")

In [4]:
emotions = ('angry','disgust','fear','happy','sad','surprise','neutral')

## fer2013

In [5]:
df = df_fer.copy( )

In [8]:
print(df.shape, df.size, df.columns)

(35887, 3) 107661 Index(['emotion', 'pixels', 'Usage'], dtype='object')


In [9]:
image_size=(48,48)
pixels = df['pixels'].tolist() # Converting the relevant column element into a list for each row
width, height = 48, 48
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')] # Splitting the string by space character as a list
    face = np.asarray(face).reshape(width, height) #converting the list to numpy array in size of 48*48
    face = cv2.resize(face.astype('uint8'),image_size) #resize the image to have 48 cols (width) and 48 rows (height)
    faces.append(face.astype('float32')) #makes the list of each images of 48*48 and their pixels in numpyarray form

faces = np.asarray(faces) #converting the list into numpy array
faces = np.expand_dims(faces, -1) #Expand the shape of an array -1=last dimension => means color space
emotions = pd.get_dummies(df['emotion']).to_numpy() #doing the one hot encoding type on emotions

In [10]:
print(faces[0].shape)
print(faces.shape)

(48, 48, 1)
(35887, 48, 48, 1)


#### Standardization

In [11]:
# normalize (from 0 to 1)
x = faces.astype('float32')
x = x / 255.0

# scale pixel value (from -1 to 1)
x = x - 0.5
x = x * 2.0

In [12]:
emotions[0]

array([1, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [14]:
q,t = emotions.shape
q = len(x)
q_train = int((0.8) * q)

## train

In [31]:
# Training data
x_train = x[:q_train]
y_train = emotions[:q_train]

# Validation data
x_val = x[q_train:]
y_val = emotions[q_train:]

data_train = (train_x, train_y)
data_val = (val_x, val_y)

In [32]:
print("\n")
print('train_x.shape (pixels): ',train_x.shape)  # ==> 4 dims -  no of images , width , height , color
print("\n")
print('train_y.shape: (labels): ',train_y.shape)
print("\n")
print('val_x.shape (pixels): ',val_x.shape)
print("\n")
print('val_y.shape: (labels): ',val_y.shape)
print("\n")



train_x.shape (pixels):  (28709, 48, 48, 1)


train_y.shape: (labels):  (28709, 7)


val_x.shape (pixels):  (7178, 48, 48, 1)


val_y.shape: (labels):  (7178, 7)




In [33]:
train_x[0].shape

(48, 48, 1)

## from 2_model

In [105]:

# LOW REPO:
    #https://pub.towardsai.net/step-by-step-guide-in-creating-your-own-emotion-recognition-system-b8aba98134c8
    #https://github.com/Nabeel110/Emotion-Recognition
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# importing tensorflow model libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
import json
from keras import backend as K
import time


In [40]:
# tuned_model
def base_model():
    model = Sequential()
    input_shape = (48,48,1)
    #1st convolution layer
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    #2nd convolution layer
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #3rd convolution layer
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    my_optimiser = tf.keras.optimizers.Adam(
    learning_rate = 0.001, beta_1=0.9, beta_2=0.999, 
        epsilon=1e-07, amsgrad=False, name='Adam')
    
    model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=my_optimiser)
    
    print(model.summary)
    
    return model

In [41]:
model_1 = base_model()

model_1.fit(x_train, y_train, 
            validation_data=(x_val, y_val), 
            epochs=15,
            verbose=1, 
            batch_size=50)

model_1.summary()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f8c75cb37c0>>
Epoch 1/15
575/575 [==============================] - 1361s 2s/step - loss: 1.7065 - accuracy: 0.3354 - val_loss: 1.5526 - val_accuracy: 0.4040
Epoch 2/15
575/575 [==============================] - 1283s 2s/step - loss: 1.3771 - accuracy: 0.4708 - val_loss: 1.2891 - val_accuracy: 0.5029
Epoch 3/15
575/575 [==============================] - 1252s 2s/step - loss: 1.2316 - accuracy: 0.5315 - val_loss: 1.1695 - val_accuracy: 0.5524
Epoch 4/15
575/575 [==============================] - 1242s 2s/step - loss: 1.1585 - accuracy: 0.5588 - val_loss: 1.1610 - val_accuracy: 0.5532
Epoch 5/15
575/575 [==============================] - 1236s 2s/step - loss: 1.1076 - accuracy: 0.5777 - val_loss: 1.1464 - val_accuracy: 0.5584
Epoch 6/15
575/575 [==============================] - 1227s 2s/step - loss: 1.0635 - accuracy: 0.5981 - val_loss: 1.0601 - val_accuracy: 0.6007
Epoch 7/15
575/575 [====

In [96]:
history = model_1

In [91]:
model_1.save("../src/model_v1_epoch15k.hdf5")

In [44]:
model_1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_20 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 24, 24, 128)      

In [98]:
scores = model_1.evaluate(x_val, y_val, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

225/225 [==============================] - 83s 368ms/step - loss: 1.0097 - accuracy: 0.6475
Accuracy: 64.75%
Test loss: 1.0096663236618042
Test accuracy: 0.6475341320037842


In [108]:
filepath = os.path.join("../src/model_v1_epoch15.hdf5")

checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max')

In [109]:
src_dir = Path.cwd().parent/'src'

In [110]:
filepath_1 = src_dir/'model_v1_epoch15_2.hdf5'

In [111]:
checkpoint_1 = ModelCheckpoint(
    filepath_1,
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max')

In [112]:
callbacks_1 = [checkpoint_1]

In [115]:
name_1 = 'model_v1_epoch15_2.hdf5'

In [122]:
import json

model_json = model_1.to_json()
with open(name_1+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model_1.save_weights(name_1)

In [118]:
model_1.load_weights("../src/model_v1_epoch15k.hdf5")

In [97]:
"""
filepath='Checkpoint_{epoch:02d}_{val_accuracy:.2f}'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
"""

"\nfilepath='Checkpoint_{epoch:02d}_{val_accuracy:.2f}'\ncheckpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')\ncallbacks_list = [checkpoint]\n"

In [99]:
"""
moment = time.localtime()
name='Model_{}_{}-{}-{}.h5'.format(model_1.history.history["val_accuracy"][-1],moment[2],moment[3],moment[4])
model_1.save(model_v1_epoch15.hdf5)
"""

'\nmoment = time.localtime()\nname=\'Model_{}_{}-{}-{}.h5\'.format(model_1.history.history["val_accuracy"][-1],moment[2],moment[3],moment[4])\nmodel_1.save(model_v1_epoch15.hdf5)\n'

In [100]:
"""
import json

model_json = model_1.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model_1.save_weights(name)
"""

'\nimport json\n\nmodel_json = model_1.to_json()\nwith open(name+\'.json\', "w") as json_file:\n    json.dump(model_json, json_file)\n\nmodel_1.save_weights(name)\n'

In [69]:
"""
# If you want to train the same model or try other models, go for this


filepath_1 = src_dir/'model_v1_epoch15.hdf5

checkpoint = keras.callbacks.ModelCheckpoint(filepath_1,
                                             monitor='val_acc',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='max')
callbacks = [checkpoint]
# if mode == "train":
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
nb_train_samples = 28709
nb_validation_samples = 3589
epochs = 150
model_info = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=epochs,
            callbacks = callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size)

# plot_model_history(model_info)
# model.save_weights('model.h5')

"""

'\n# If you want to train the same model or try other models, go for this\n\n\nfilepath_1 = src_dir/\'model_v1_epoch15.hdf5\n\ncheckpoint = keras.callbacks.ModelCheckpoint(filepath_1,\n                                             monitor=\'val_acc\',\n                                             verbose=1,\n                                             save_best_only=True,\n                                             mode=\'max\')\ncallbacks = [checkpoint]\n# if mode == "train":\nmodel.compile(loss=\'categorical_crossentropy\',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=[\'accuracy\'])\nnb_train_samples = 28709\nnb_validation_samples = 3589\nepochs = 150\nmodel_info = model.fit_generator(\n            train_generator,\n            steps_per_epoch=nb_train_samples // batch_size,\n            epochs=epochs,\n            callbacks = callbacks,\n            validation_data=validation_generator,\n            validation_steps=nb_validation_samples // batch_size)\n\n# plot_model_history(mod

## test

In [ ]:
model_1